Reference:

Offline Handwritten Text OCR
https://www.kaggle.com/code/aman10kr/offline-handwritten-text-ocr

In [ ]:
!pip install imutils
!pip install np_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 386.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56439 sha256=9594ab18c9b787e78835a43cca4964af42ee32d927decbf868041db026aaeabe
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils


In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import os
import random
import cv2
import imutils
import random
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import csv
from google.colab import drive
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical
from keras.src.utils import np_utils
from keras.models import Sequential
from keras import optimizers
from sklearn.preprocessing import LabelBinarizer
from keras import backend as K
from keras.layers import Dense, Activation, Flatten, Dense,MaxPooling2D, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

In [ ]:
# Mount Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
train_dir = "/content/gdrive/My Drive/train"
train_data = []
img_size = 32
phishing_keywords = [".xyz", ".pro", "klaim", "bonus", "promo", ".net", ".site", ".fyi"]
for Phishing in os.listdir(train_dir):
    if Phishing in phishing_keywords:
        continue
    count = 0
    sub_directory = os.path.join(train_dir, Phishing)
    for UnPhishing in os.listdir(sub_directory):
        count+=1
        if count > 4000:
            break
        img = cv2.imread(os.path.join(sub_directory, UnPhishing),0)
        img = cv2.resize(img,(img_size,img_size))
        train_data.append([img, Phishing])

In [ ]:
len(train_data)

786

In [ ]:
validation_dir = "/content/gdrive/My Drive/validation"
validation_data = []
img_size = 32
phishing_keywords = [".xyz", ".pro", "klaim", "bonus", "promo", ".net", ".site", ".fyi"]
for Phishing in os.listdir(validation_dir):
    if Phishing in phishing_keywords:
        continue
    count = 0
    sub_directory = os.path.join(validation_dir, Phishing)
    for UnPhishing in os.listdir(sub_directory):
        count+=1
        if count > 4000:
            break
        img = cv2.imread(os.path.join(sub_directory, UnPhishing),0)
        img = cv2.resize(img,(img_size,img_size))
        validation_data.append([img, Phishing])

In [ ]:
len(validation_data)

99

In [ ]:
test_dir = "/content/gdrive/My Drive/validation"
test_data = []
img_size = 32
phishing_keywords = [".xyz", ".pro", "klaim", "bonus", "promo", ".net", ".site", ".fyi"]
for Phishing in os.listdir(test_dir):
    if Phishing in phishing_keywords:
        continue
    count = 0
    sub_directory = os.path.join(test_dir, Phishing)
    for UnPhishing in os.listdir(sub_directory):
        count+=1
        if count > 4000:
            break
        img = cv2.imread(os.path.join(sub_directory, UnPhishing),0)
        img = cv2.resize(img,(img_size,img_size))
        test_data.append([img, Phishing])

In [ ]:
len(test_data)

99

In [ ]:
random.shuffle(train_data)
random.shuffle(validation_data)
random.shuffle(test_data)

In [ ]:
train_X = []
train_Y = []
for features,label in train_data:
    train_X.append(features)
    train_Y.append(label)

In [ ]:
validation_X = []
validation_Y = []
for features,label in validation_data:
    validation_X.append(features)
    validation_Y.append(label)

In [ ]:
test_X = []
test_Y = []
for features,label in test_data:
    test_X.append(features)
    test_Y.append(label)

In [ ]:
LB = LabelBinarizer()
train_Y = LB.fit_transform(train_Y)
validation_Y = LB.fit_transform(validation_Y)
test_Y = LB.fit_transform(test_Y)

In [ ]:
train_X = np.array(train_X)/255.0
train_X = train_X.reshape(-1,32,32,1)
train_Y = np.array(train_Y)

In [ ]:
validation_X = np.array(validation_X)/255.0
validation_X = validation_X.reshape(-1,32,32,1)
validation_Y = np.array(validation_Y)

In [ ]:
test_X = np.array(test_X)/255.0
test_X = test_X.reshape(-1,32,32,1)
test_Y = np.array(test_Y)

In [ ]:
print(train_Y.shape, validation_Y.shape, test_Y.shape)

(786, 1) (99, 1) (99, 1)


In [ ]:
# Assuming train_Y, validation_Y, and test_Y are your label data
train_Y_one_hot = to_categorical(train_Y, num_classes=1000)
validation_Y_one_hot = to_categorical(validation_Y, num_classes=100)
test_Y_one_hot = to_categorical(test_Y, num_classes=100)

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape=(32,32,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(35, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 32, 32, 32)        320       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 7, 7, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 5, 5, 128)         73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 2, 2, 128)        

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [ ]:
# Fitting the model
history = model.fit(train_X, train_Y_one_hot, epochs=50, batch_size=32, validation_data=(validation_X, validation_Y_one_hot), verbose=1)

Epoch 1/50


ValueError: ignored

In [ ]:
LB = LabelBinarizer()
train_Y = LB.fit_transform(train_Y)
validation_Y = LB.transform(validation_Y)  # Use transform for validation set
test_Y = LB.transform(test_Y)  # Use transform for test set

# Convert lists to NumPy arrays
train_Y_one_hot = to_categorical(train_Y, num_classes=35)
validation_Y_one_hot = to_categorical(validation_Y, num_classes=35)
test_Y_one_hot = to_categorical(test_Y, num_classes=35)

/usr/local/lib/python3.10/dist-packages/numpy/lib/arraysetops.py:612: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
# Print shapes
print("Training data shapes:", train_X.shape, train_Y_one_hot.shape)
print("Validation data shapes:", validation_X.shape, validation_Y_one_hot.shape)

Training data shapes: (786, 32, 32) (786, 35)
Validation data shapes: (99, 32, 32) (99, 35)


In [ ]:
history = model.fit(train_X, train_Y_one_hot, epochs=50, batch_size=32, validation_data=(validation_X, validation_Y_one_hot), verbose=1)

ValueError: ignored

In [ ]:
test_loss, test_accuracy = model.evaluate(test_X, test_Y_one_hot)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training Accuracy vs Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training Loss vs Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Recognition and Post-Processing

In [ ]:
def sort_contours(cnts, method="left-to-right"):
    reverse = False
    i = 0
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
    key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

In [ ]:
def get_letters(img):
    letters = []
    image = cv2.imread(img)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret,thresh1 = cv2.threshold(gray ,127,255,cv2.THRESH_BINARY_INV)
    dilated = cv2.dilate(thresh1, None, iterations=2)

    cnts = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sort_contours(cnts, method="left-to-right")[0]
    # loop over the contours
    for c in cnts:
        if cv2.contourArea(c) > 10:
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        roi = gray[y:y + h, x:x + w]
        thresh = cv2.threshold(roi, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        thresh = cv2.resize(thresh, (32, 32), interpolation = cv2.INTER_CUBIC)
        thresh = thresh.astype("float32") / 255.0
        thresh = np.expand_dims(thresh, axis=-1)
        thresh = thresh.reshape(1,32,32,1)
        ypred = model.predict(thresh)
        ypred = LB.inverse_transform(ypred)
        [x] = ypred
        letters.append(x)
    return letters, image

#plt.imshow(image)

In [ ]:
def get_word(letter):
    word = "".join(letter)
    return word

In [ ]:
letters,image = get_letters("/content/gdrive/My Drive/train/Phishing/Phish (316).jpg")
#Print the recognized word
word = ''.join(letters)
print(word)
plt.imshow(image)

In [ ]:
# Display the image with bounding boxes
cv2.imshow('Image with Bounding Boxes', image)

In [ ]:
def get_letters(img):
    letters = []
    image = cv2.imread(img)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret,thresh1 = cv2.threshold(gray ,127,255,cv2.THRESH_BINARY_INV)
    dilated = cv2.dilate(thresh1, None, iterations=2)

    cnts = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sort_contours(cnts, method="left-to-right")[0]
    # loop over the contours
    for c in cnts:
        if cv2.contourArea(c) > 10:
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        roi = gray[y:y + h, x:x + w]
        thresh = cv2.threshold(roi, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        thresh = cv2.resize(thresh, (32, 32), interpolation = cv2.INTER_CUBIC)
        thresh = thresh.astype("float32") / 255.0
        thresh = np.expand_dims(thresh, axis=-1)
        thresh = thresh.reshape(1,32,32,1)
        ypred = model.predict(thresh)
        ypred = LB.inverse_transform(ypred)
        [x] = ypred
        letters.append(x)
    return letters, image

#plt.imshow(image)